In [1]:
#pip install --pre mesa[viz]

In [2]:
# import libraries
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.animation import FuncAnimation
import mesa
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import numpy as np
import math
from IPython.display import HTML
from matplotlib import rc


This code runs with mesa version 3.00 or higher

In [3]:
# check mesa version
print("Mesa version: ", mesa.__version__)

Mesa version:  3.1.1


In [4]:
# Function to calculate Euclidean distance (distance between two points)
def euclidean_distance(pos1, pos2):
    # Use the formula to calculate distance between two coordinates
    return math.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2)

# Base Model

In [5]:
###################
### AGENT CLASS ###
###################
### I'm defining an agent class for the crew that are essentially walking signs
class CrewMember(Agent):
    def __init__(self, model, vision=5):
        super().__init__(model)
        self.previous_pos = None  # Keep track of previous position
        self.vision = vision # vision is range to detect a NavigationAgent
        self.type = 'Crew' ### add an atrivute to identify crew or passenger
    
    def move_randomly(self):
        #check if passenger (navigationAgent) is in vision range
        vision_area = self.model.grid.get_neighborhood(self.pos, moore=True, radius=self.vision, include_center=False)
        for cell in vision_area:
            for i in self.model.grid.get_cell_list_contents(cell): # i of list of agents in the given cell, that is in vision ares (defined in vision_area)
                if isinstance(i,NavigationAgent):
                    return  #stop moving if see passenger 
                    
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        # Filter steps to only those without obstacles and with fewer than 8 agents
        valid_steps = [step for step in possible_steps if step not in self.model.obstacles and len(self.model.grid.get_cell_list_contents(step)) < 8]
        if valid_steps:
            random_step = self.random.choice(valid_steps) # Randomly choose a valid position and move there
            self.model.grid.move_agent(self, random_step) # MESA `move_agent` function moves the agent to the chosen position

    def step(self):
        self.move_randomly()

# Define the NavigationAgent class
class NavigationAgent(Agent):
    def __init__(self, model, vision=5): # Default vision range of 5 cells
        super().__init__(model)  # MESA `Agent` class initialization, auto-assigns unique_id in Mesa 3.0
        # Attributes of each agent
        self.found_exit = False  # Track if agent has reached the exit
        self.previous_pos = None  # Previous position of the agent
        self.vision = vision  # Vision range of the agent
        self.type= 'Passenger'

    # Function to move the agent towards the exit
    def move_towards_exit(self):
        self.previous_pos = self.pos  # Store the current position before moving
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        
        min_distance = float('inf')  # Start with a very large distance
        best_step = None  # Initialize best step as None
        
        # Find the closest exit and the best step toward it
        for step in possible_steps:
            if step not in self.model.obstacles and len(self.model.grid.get_cell_list_contents(step)) < 8:
                for exit_location in self.model.exit_locations:
                    dist = euclidean_distance(step, exit_location)  # Distance to the exit
                    if dist < min_distance:
                        min_distance = dist
                        best_step = step  # Update best step to be closer to the exit
        
        if best_step:
            self.model.grid.move_agent(self, best_step)  # Move the agent to the best step


    # Function to move the agent randomly if the exit is not in sight
    def move_randomly(self):
        self.previous_pos = self.pos
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        
        # Filter steps to only those without obstacles and with fewer than 8 agents
        valid_steps = [step for step in possible_steps if step not in self.model.obstacles and len(self.model.grid.get_cell_list_contents(step)) < 8]
        
        if valid_steps:
            # Randomly choose a valid position and move there
            random_step = self.random.choice(valid_steps)
            # MESA `move_agent` function moves the agent to the chosen position
            self.model.grid.move_agent(self, random_step)

    # Define the actions the agent will take in each step
    def step(self):
        ### If the agent is at one of  the exit, mark as exited
        if self.pos in self.model.exit_locations:
            self.found_exit = True  # Set the agent's exit status to True
            self.model.grid.remove_agent(self)  # MESA function to remove the agent from the grid
          #  self.model.schedule.remove(self) ###remove from scheduler
            self.remove()  #self.remove() to remove from AgentSet
            self.model.cumulative_exited += 1  # Count this agent in cumulative exited agents
        else:
            # MESA `get_neighborhood` checks the agent's vision area for the exit
            vision_area = self.model.grid.get_neighborhood(self.pos, moore=True, radius=self.vision, include_center=False)
            exit_in_vision = any(exit_location in vision_area for exit_location in self.model.exit_locations)
            
            # Move towards the exit if it's in sight, otherwise move randomly
            if exit_in_vision:
                self.move_towards_exit()
            else:
                self.move_randomly()

###################
### MODEL CLASS ###
###################

# Define the model class to handle the overall environment
class FloorPlanModel(Model):
    def __init__(self, width, height, num_agents, num_crew, agent_vision): 
        super().__init__()  # `Model` class initialization
        
        # Basic model settings
        self.num_agents = num_agents
        self.num_crew = num_crew
        self.agent_vision = agent_vision
        self.grid = MultiGrid(width, height, False)  # MESA grid with dimensions; False means no wrapping
        self.exit_locations = [(9, 0), (21, 0), (33, 0), (45, 0), (9, 27), (21, 27), (33, 27), (45, 27), (111, 0), (123, 0), (135, 0), (147, 0), (111, 27), (123, 27), (135, 27), (147, 27)]  ### List of multiple exit locations
        self.obstacles = [(5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (5, 15), (5, 16), (5, 17), (5, 18), (5, 19), (6, 10), (6, 11), (6, 12), (6, 13), (6, 14), (6, 15), (6, 16), (6, 17), (6, 18), (6, 19), (7, 10), (7, 11), (7, 12), (7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (8, 10), (8, 11), (8, 12), (8, 13), (8, 14), (8, 15), (8, 16), (8, 17), (8, 18), (8, 19), (9, 10), (9, 11), (9, 12), (9, 13), (9, 14), (9, 15), (9, 16), (9, 17), (9, 18), (9, 19), (10, 10), (10, 11), (10, 12), (10, 13), (10, 14), (10, 15), (10, 16), (10, 17), (10, 18), (10, 19)]
        self.signs = [(15, 15), (20, 20), (25, 25)]
         
        self.cumulative_exited = 0 # Initialize cumulative exited count
        self.agentss = [] #make list with amount of agents

        # Initialize DataCollector (MESA tool for tracking metrics across steps)
        self.datacollector = DataCollector(
            model_reporters={
                "Active Agents": lambda m: len(m.agents),  # Count of agents still active
                "Exited Agents": lambda m: sum(1 for agent in m.agents if isinstance(agent, NavigationAgent) and agent.found_exit),
                "Cumulative Exited Agents": lambda m: m.cumulative_exited,  # Cumulative exited count
                "Agents per Cell": self.count_agents_per_cell  # Counts agents in each cell
            },
            agent_reporters={
                "Found Exit": lambda a: a.found_exit if isinstance(a, NavigationAgent) else None  # Reports exit status per agent
            }
        )

        self.place_agents(agent_vision)  # Place agents on the grid
        self.datacollector.collect(self) # Collect data at the start of the simulation
        

    ### Function to place agents on particular entry points, these points will be the doors to top deck and the stairs
    def place_agents(self, agent_vision):
        #### Define the fixed entries (spawn points) of crew and others
        crewrooms = [(122,10),(122,19), (134,10),(134,19), (148,10),(148,19)]  #width and height!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        num_crewrooms = len(crewrooms)
        entries = [(7, 9), (7, 20), (109, 20), (109, 7)] 
        num_entries = len(entries)
    
        for i in range(self.num_agents):
            agent = NavigationAgent(self, vision=agent_vision)  # Create agent with the specified vision ###changed self to i why, idk?
            spawn_agent = entries[i % num_entries]  # Alternate between the spawn points
            cell_contents = self.grid.get_cell_list_contents(spawn_agent)
            if len(cell_contents) < 8:
                self.grid.place_agent(agent, spawn_agent)  # Place the agent at the spawn point, only with <8 agents incell
                self.agentss.append(agent) #create passenger agent (blue)

        for i in range(self.num_crew):
            crew = CrewMember(self, vision=agent_vision)  #its not referring to i but to 'self' so idk if thats the way
            spawn_crew = crewrooms[i % num_crewrooms]
            cell_contents = self.grid.get_cell_list_contents(spawn_crew)
            if len(cell_contents) < 8:
                self.grid.place_agent(crew, spawn_crew)  # Place the agent at the spawn point, only with <8 agents incell
                self.agentss.append(crew) #create crew agent (red)

    # Function to count agents in each cell
    def count_agents_per_cell(self):
        agent_counts = {}  # Dictionary to store agent counts by cell position
        # MESA `coord_iter` function iterates over grid cells and their contents
        for cell in self.grid.coord_iter():
            cell_contents, (x, y) = cell  # Unpack cell contents and coordinates
            # Count NavigationAgents in each cell
            nav_agent_count = sum(1 for obj in cell_contents if isinstance(obj, NavigationAgent))
            if nav_agent_count > 0:
                agent_counts[(x, y)] = nav_agent_count
        return agent_counts
    
    # Function to get the grid data for visualization
    def get_grid(self):
        # 0: empty, 1: obstacle, 2: agent, 3: exit, 4: sign
        grid_data = np.zeros((self.grid.height, self.grid.width))
        
        # Mark obstacles on the grid
        for x, y in self.obstacles:
            grid_data[y, x] = 1
        
        # Mark agents on the grid
        for agent in self.agents:
            if isinstance(agent, NavigationAgent):
                x, y = agent.pos
                grid_data[y, x] = 2
        
        #### Mark signs and exits, multiple
        for x, y in self.signs:
            grid_data[y, x] = 4
        for exit_x,exit_y in self.exit_locations:
            grid_data[exit_y,exit_x]=16      #3 exit locations, if there are more change!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
       # exit_x, exit_y = self.exit_locations
        grid_data[exit_y, exit_x] = 16
        return grid_data

    # Model step function to update the simulation
    def step(self):
        self.agents.do("step")  # MESA 3.0 function to execute the `step` function of each agent
        self.datacollector.collect(self)  # MESA DataCollector collects metrics at each step

    

In [6]:
# Run the model and see the results
model = FloorPlanModel(160, 28, 2, 15, 5) # run the model with 30x30 grid and 100 agents and a vision of 5 because more agents its trippinnnn
for i in range(100): # run the model for 100 steps
    model.step() # step the model by 1

# Collect the data from the model
model_data = model.datacollector.get_model_vars_dataframe()
agent_data = model.datacollector.get_agent_vars_dataframe()


In [7]:
model_data # print the model data

,Active Agents,Exited Agents,Cumulative Exited Agents,Agents per Cell
0,17,0,0,"{(7, 9): 1, (7, 20): 1}"
1,17,0,0,"{(7, 21): 1, (8, 8): 1}"
2,17,0,0,"{(6, 20): 1, (9, 9): 1}"
3,17,0,0,"{(6, 21): 1, (8, 9): 1}"
4,17,0,0,"{(5, 21): 1, (9, 9): 1}"
...,...,...,...,...
96,16,0,1,"{(0, 15): 1}"
97,16,0,1,"{(1, 14): 1}"
98,16,0,1,"{(0, 14): 1}"
99,16,0,1,"{(1, 15): 1}"


In [8]:
agent_data # print the agent data

Found Exit
Step AgentID           
0    1            False
     2            False
     3             None
     4             None
     5             None
...                 ...
100  13            None
     14            None
     15            None
     16            None
     17            None

[1632 rows x 1 columns]

# Visualization with User Interface
You don't have to understand every single line in the visualisation code below. Please understand the code to extend that you can introduce changes to it when needed

In [9]:
# Visualization Function
def plot_grid(model, ax):
    rc("animation", embed_limit=700)  # Set a higher limit in MB to allow smoother animation playback
    grid_data = model.get_grid()  # Retrieve the current state of the grid from the model
    ax.clear()  # Clear any previous plots on the axes to prevent overlap in visualizations
    
    # Define color mappings:
    # 0 (empty) -> white, 1 (obstacle) -> black, 2 (agent) -> blue,
    # 3 (exit) -> green, 4 (sign) -> orange
    cmap = mcolors.ListedColormap(['white', 'black', 'blue', 'green', 'orange'])
    bounds = [0, 1, 2, 3, 4]  # Boundaries to separate each category
    norm = mcolors.BoundaryNorm(bounds, cmap.N)  # Normalizes values to assign colors to each category
    
    # Display the grid data with color mapping applied.
    # Setting 'origin' to 'lower' places the (0,0) coordinate at the bottom-left.
    ax.imshow(grid_data, cmap=cmap, norm=norm, origin='lower')
    
    # Add grid lines for better cell visibility
    ax.grid(which='both', color='gray', linestyle='-', linewidth=2)
    
    # Customize grid display: set grid to start from -0.5 with labels at intervals of 1
    # Set minor ticks for cell boundaries, with thicker and darker lines
    ax.set_xticks(np.arange(-0.5, model.grid.width, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, model.grid.height, 1), minor=True)
    ax.grid(which='minor', color='gray', linestyle='-', linewidth=1.5)  # Thicker, darker lines for cell boundaries

    # Set major ticks for labels at intervals of 5, with lighter and thinner lines
    ax.set_xticks(np.arange(0, model.grid.width, 5), minor=False)
    ax.set_yticks(np.arange(0, model.grid.height, 5), minor=False)
    ax.grid(which='major', color='lightgray', linestyle='-', linewidth=0)  # invisible line
    
    ### Label the exit point on the grid in red text for easy identification
    for exit_x, exit_y in model.exit_locations:
        ax.text(exit_x, exit_y, 'EXIT', ha='center', va='center', fontsize=12, color='red', fontweight='bold')
    # Add labels for each sign location in black text for visibility
    for sign_x,sign_y in model.signs:
        ax.text(sign_x, sign_y, 'SIGN', ha='center', va='center', fontsize=10, color='black', fontweight='bold')

    # Set the title to show the current step number in the model
    ax.set_title(f"Step {model.steps}", fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=7)  # Control tick label size
    ####
    # Draw movement arrows for agents to show the direction they are traveling
    for agent in model.agents:
        if isinstance(agent, NavigationAgent):
            ax.scatter(agent.pos[0], agent.pos[1], color='blue', s=40)  # Blue for passenger agents
        elif isinstance(agent, CrewMember):
            ax.scatter(agent.pos[0], agent.pos[1], color='red', s=40)  # Red for crew agents

    # Draw movement arrows for agents to show the direction they are traveling
    for agent in model.agents:
        if isinstance(agent, NavigationAgent) and agent.previous_pos:
            start_x, start_y = agent.previous_pos  # Previous position of the agent
            end_x, end_y = agent.pos  # Current position of the agent
            
            # Draw an arrow from the previous position to the current position
            ax.arrow(
                start_x, start_y,
                end_x - start_x, end_y - start_y,
                head_width=0.3, head_length=0.3, fc='yellow', ec='yellow'  # Yellow arrow for movement direction
            )
    ####        

# Animation Update Function
def update(frame, model, ax):
    # For every frame, update the model state if it's not the first frame
    if frame > 0:
        model.step()  # Run one step of the model simulation
    plot_grid(model, ax)  # Redraw the grid with updated agent positions
    
# Run the Animation with a larger figure size
def run_animation(model, steps):
    fig, ax = plt.subplots(figsize=(15, 5))  # Create a 10x10 figure for the plot!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    plot_grid(model, ax)  # Plot the initial grid state
    # Create an animation that updates the grid for each step
    anim = FuncAnimation(fig, update, frames=steps+1, fargs=(model, ax), repeat=False)
    plt.close(fig)  # Close the figure after animation creation to avoid duplicate displays
    return anim

# Your imports and function definitions remain the same, up until where you initialize and run the model
import ipywidgets as widgets  # For interactive widgets (slider, button)
from IPython.display import display, HTML  # To display widgets and HTML animations
import time  # For tracking elapsed time

# Slider to choose the number of agents
agent_slider = widgets.IntSlider(
    value=10,      # Default starting number of agents
    min=10,        # Minimum number of agents allowed
    max=500,       # Maximum number of agents allowed
    step=10,       # Step size for slider increments
    description='Num Agents:',  # Label for the slider
    continuous_update=False     # Only update value when slider is released
)

# Slider to choose the number of time steps (frames) for the animation
time_step_slider = widgets.IntSlider(
    value=50,      # Default starting number of steps
    min=1,         # Minimum time steps allowed
    max=500,       # Maximum time steps allowed
    step=1,        # Step size for slider increments
    description='Time Steps:',  # Label for the slider
    continuous_update=False     # Only update value when slider is released
)

# Slider to choose the vision range (vision radius) of agents
vision_slider = widgets.IntSlider(
    value=5,      # Default starting vision range
    min=1,        # Minimum vision range
    max=20,       # Maximum vision range
    step=1,       # Step size for slider increments
    description='Vision:',  # Label for the slider
    continuous_update=False     # Only update value when slider is released
)

# Button to start the simulation with current slider settings
run_button = widgets.Button(description="Run Simulation")

# Output widget for displaying the animation and elapsed time
output_widget = widgets.Output()

# Label to show elapsed time during the simulation
elapsed_time_label = widgets.Label(value="Elapsed time: 0.0 seconds")

# Flag variable to control timing function
stop_timer = False

# Display the interface with sliders, button, label, and output area
display(agent_slider, time_step_slider, vision_slider, run_button, elapsed_time_label, output_widget)

# Define the function to initialize and run the model
def run_model(change):
    global stop_timer, model_data, agent_data  # Allow variables to be accessed globally
    with output_widget:  # Use output widget to display the animation
        output_widget.clear_output()  # Clear any previous output
        num_agents = agent_slider.value  # Get the number of agents from the slider
        time_steps = time_step_slider.value  # Get the number of steps from the slider
        agent_vision = vision_slider.value  # Get the vision range from the slider
        model = FloorPlanModel(width=160, height=28, num_agents=num_agents, num_crew=2, agent_vision=agent_vision)  # Initialize the model!!!!!!!!!!!


        # Reset timer flag and start timing for the animation
        stop_timer = False
        start_time = time.time()

        def update_time_label():
            while not stop_timer:  # Keep updating time label until timer stops
                elapsed_time = time.time() - start_time
                elapsed_time_label.value = f"Elapsed time: {elapsed_time:.1f} seconds"
                time.sleep(0.1)  # Update every 0.1 seconds for real-time effect

        # Start elapsed time tracking in a separate thread
        import threading
        timer_thread = threading.Thread(target=update_time_label, daemon=True)
        timer_thread.start()

        # Run the animation with selected number of steps
        anim = run_animation(model, steps=time_steps)

        # Display the animation output in HTML format
        output = HTML(anim.to_jshtml())
        display(output)

        # Stop the timer after the simulation completes
        stop_timer = True
        elapsed_time = time.time() - start_time
        elapsed_time_label.value = f"Total elapsed time: {elapsed_time:.1f} seconds"

        # Retrieve and display model and agent data after simulation completes
        model_data = model.datacollector.get_model_vars_dataframe()  # Data for the model over time
        agent_data = model.datacollector.get_agent_vars_dataframe()  # Data for each agent over time
    return model, model_data, agent_data  # Return model and data for further inspection

# Attach the run_model function to the run button click event
run_button.on_click(run_model)




IntSlider(value=10, continuous_update=False, description='Num Agents:', max=500, min=10, step=10)

IntSlider(value=50, continuous_update=False, description='Time Steps:', max=500, min=1)

IntSlider(value=5, continuous_update=False, description='Vision:', max=20, min=1)

Button(description='Run Simulation', style=ButtonStyle())

Label(value='Elapsed time: 0.0 seconds')

Output()

In [10]:
print("\nModel Data:")
model_data


Model Data:


,Active Agents,Exited Agents,Cumulative Exited Agents,Agents per Cell
0,17,0,0,"{(7, 9): 1, (7, 20): 1}"
1,17,0,0,"{(7, 21): 1, (8, 8): 1}"
2,17,0,0,"{(6, 20): 1, (9, 9): 1}"
3,17,0,0,"{(6, 21): 1, (8, 9): 1}"
4,17,0,0,"{(5, 21): 1, (9, 9): 1}"
...,...,...,...,...
96,16,0,1,"{(0, 15): 1}"
97,16,0,1,"{(1, 14): 1}"
98,16,0,1,"{(0, 14): 1}"
99,16,0,1,"{(1, 15): 1}"


In [11]:
print("\nAgent Data:")
agent_data.head(400)



Agent Data:


Found Exit
Step AgentID           
0    1            False
     2            False
     3             None
     4             None
     5             None
...                 ...
23   13            None
     14            None
     15            None
     16            None
     17            None

[400 rows x 1 columns]

In [12]:
#  Uncomment the code below to display the 'Agents per Cell' data for each step

#print("Agents per Cell at each step:")
#for step, agents_per_cell in model_data["Agents per Cell"].items():
#    print(f"\nStep {step}:")
#    for cell, count in agents_per_cell.items():
#        print(f"  Cell {cell}: {count} agent(s)")

In [13]:
coordinates = []
#second number should be end+1
for a in range(5, 11):  # Replace 0 and 50 with your desired range for a
    for b in range(10, 20):  # Replace 0 and 20 with your desired range for b
        coordinates.append((a, b))  # Append the coordinate pair to the list

print(coordinates)


[(5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (5, 15), (5, 16), (5, 17), (5, 18), (5, 19), (6, 10), (6, 11), (6, 12), (6, 13), (6, 14), (6, 15), (6, 16), (6, 17), (6, 18), (6, 19), (7, 10), (7, 11), (7, 12), (7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (8, 10), (8, 11), (8, 12), (8, 13), (8, 14), (8, 15), (8, 16), (8, 17), (8, 18), (8, 19), (9, 10), (9, 11), (9, 12), (9, 13), (9, 14), (9, 15), (9, 16), (9, 17), (9, 18), (9, 19), (10, 10), (10, 11), (10, 12), (10, 13), (10, 14), (10, 15), (10, 16), (10, 17), (10, 18), (10, 19)]
